# Homework 3

**Name:** Omar Alejandro Guzmán Munguía

**e-mail:** omar.guzman5063@alumnos.udg.mx

# MODULES

In [148]:
# Load modules
import numpy as np
import math

import plotly.graph_objects as go

from scipy.stats import exponweib
from scipy.spatial import distance

import pandas as pd
from scipy.stats  import wrapcauchy

# CLASSES

In [137]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Activity 1: Path length - (BM1 vs BM2 vs CRW)

- Write a function that returns a **Brownian Motion (BM)** trajectory in **pandas** df.
- Write a function that returns a **Correlated Random Walk (CRW)** trajectory in **pandas** df.
- Write a function that returns the **path length** for a given trajectory.
- **Compare** at least the **path lengt** of **three** trajectories as shown in the figure below.
- Display the results using **plotly.**

# FUNCTIONS

In [138]:
#####################################################################################
# Brownian motion trajectoy
#####################################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
    """
    Arguments:
        n_steps: Number of steps in the trajectory.
        speed: Speed of the trajectory.
        s_pos: Starting position of the trajectory.
    Returns:
        BM_2d_df:  DataFrame with the x and y positions of the trajectory.
    """
    
    # Init velocity vector
    velocity =Vec2d(speed,0)
    
    # Init DF
    BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos'])    
    # Add initial position
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])    
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
    
    # Generate the trajectory
    for i in range(n_steps-1):        
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)
    
        temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])    
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
        
    return BM_2d_df

In [139]:
#####################################################################################
# Correlated Random Walk (CRW)
#####################################################################################
def crw_trajectory(n_steps=1000, speed=5, correlation_coeff=0.9, s_pos=0.0):
    """
    Arguments:
        n_steps: Number of steps in the trajectory.
        speed: Length of each step.
        correlation_coeff: Correlation coefficient (between -1 and 1) that controls the correlation between consecutive steps.
        s_pos: Initial direction of movement in radians (default is 0.0).
    Returns:
        crw_df: A pandas DataFrame with columns ['x', 'y'] representing the trajectory.
    """
    # Initialize the trajectory with the starting point at the origin
    trajectory = [Vec2d(0.0, 0.0)]
    
    # Initialize the current direction
    current_direction = s_pos
    
    for _ in range(n_steps):
        # Generate a random angle change based on the correlation coefficient
        angle_change = np.random.normal(0, np.sqrt(1 - correlation_coeff**2))
        
        # Update the current direction
        current_direction += angle_change
        
        # Calculate the next step vector
        next_step = Vec2d(speed, 0).rotated(current_direction)
        
        # Update the trajectory
        next_position = trajectory[-1] + next_step
        trajectory.append(next_position)
    
    # Convert the trajectory to a pandas DataFrame
    crw_df = pd.DataFrame([(pos.x, pos.y) for pos in trajectory], columns=['x', 'y'])
    
    return crw_df

In [140]:
#####################################################################################
# Path length
#####################################################################################
def path_length(trajectory_df, s_pos=[0]):
    """
    Arguments:
        trajectory_df: DataFrame with the x and y positions of the trajectory.
        s_pos: Starting position of the trajectory.
    Returns:
        path_length: Length of the trajectory.
    """
    # Initialize the path length
    path_length = pd.DataFrame(columns=['y_pos'])
    df = pd.DataFrame([{'y_pos':s_pos[0]}])

    path_length = pd.concat([path_length, df], ignore_index=True)

    for i in range(len(trajectory_df)-1):
        df = pd.DataFrame([{'y_pos':path_length.y_pos[i] + distance.euclidean(trajectory_df.iloc[i], trajectory_df.iloc[i+1])}])
        path_length = pd.concat([path_length, df], ignore_index=True)
        
    return path_length

In [141]:
# Generate trajectories
bm_traj_3 = bm_2d(speed=3)
bm_traj_6 = bm_2d(speed=6)
crw_traj_6 = crw_trajectory(speed=6)

# Calculate path lengths
bm_pl_3 = path_length(bm_traj_3)
bm_pl_6 = path_length(bm_traj_6)
crw_pl_6 = path_length(crw_traj_6)

In [142]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x= bm_pl_3.index,
    y= bm_pl_3.y_pos,
    name='Path length - BM 3',
    mode='lines',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x= bm_pl_6.index,
    y= bm_pl_6.y_pos,
    name='Path length - BM 6',
    marker = dict(size=2),
    line = dict(width=6),
    mode='lines',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x= crw_pl_6.index,
    y= crw_pl_6.y_pos,
    name='Path length - CRW 6',
    mode='lines',
    showlegend=True
))

fig.update_layout(
    title='Path Lengths',
    height=800,
)

fig.show()

# Activity 2: Mean Squared Displacement - (BM vs CRW)

- Write a function that returns the **mean squared displacement** for a given trajectory.
- Compare the **mean squared displacement** curves of at least two trajectories of
different kinds, as shown in the figure below.
- Display the results using **plotly.**

# FUNCTIONS

In [167]:
#####################################################################################
# Mean Squared Displacement (MSD)
#####################################################################################
def msd(trajectory_df):
    """    
    Arguments:
        trajectory_df: DataFrame with columns 
    Returns:
        msd_df: DataFrame with columns ['tau', 'msd'] (time lag vs MSD).
    """
    x_col = 'x' if 'x' in trajectory_df.columns else 'x_pos'
    y_col = 'y' if 'y' in trajectory_df.columns else 'y_pos'
    
    # Extract coordinates
    x = trajectory_df[x_col].values
    y = trajectory_df[y_col].values
    n = len(x)
    
    msd_values = np.zeros(n - 1)  # Store MSD values
    taus = np.arange(1, n)  # All possible time lags
    
    for tau in taus:
        displacements = (x[tau:] - x[:-tau])**2 + (y[tau:] - y[:-tau])**2
        msd_values[tau - 1] = np.mean(displacements)  # Store MSD value for this tau
    
    return pd.DataFrame({'tau': taus, 'msd': msd_values})  

In [168]:
# Calculate MSDs
msd_bm = msd(bm_traj_6)
msd_crw = msd(crw_traj_6)

fig = go.Figure()
fig.add_trace(go.Scatter(x=msd_bm['tau'], y=msd_bm['msd'], mode='lines', name='MSD BM6'))
fig.add_trace(go.Scatter(x=msd_crw['tau'], y=msd_crw['msd'], mode='lines', name='MSD CRW6 c=0.9'))

fig.update_layout(
    title='Mean Squared Displacement - (BM vs CRW)',
    xaxis_title='Time',
    yaxis_title='MSD',
    hovermode='x unified',
    height=600
)
fig.show()

# Activity 3: Turning-angle Distribution - (source dist. vs observed dist.)

- Consider two **CRW** trajectories with different **Cauchy coefficients.**
- Write a function that returns the **turning angles** for a given trajectory.
- Compare the observed distribution **(histogram)** to the source distribution (**curve**) for
both trajectories, as shown in the figure below.
- Display the results using **plotly.**

# FUNCTIONS

In [179]:
#####################################################################################
# Turning angles
#####################################################################################
def turning_angles(trajectory_df):
    """
    Arguments:
        trajectory_df: DataFrame with the x and y positions of the trajectory.
    Returns:    
        angles_df: DataFrame with the turning angles.
    """
    
    # Extract coordinates
    x = trajectory_df['x'].values
    y = trajectory_df['y'].values
    
    # Calculate displacement vectors
    dx = np.diff(x)
    dy = np.diff(y)
    
    # Calculate turning angles (angle between consecutive displacement vectors)
    turning_angles = np.arctan2(dy[1:], dx[1:]) - np.arctan2(dy[:-1], dx[:-1])
    
    turning_angles = np.arctan2(np.sin(turning_angles), np.cos(turning_angles))
    
    return turning_angles

In [180]:
#####################################################################################
# Plotting turning angle distribution
#####################################################################################
def plot_turning_angle_distribution(trajectory_df, cauchy_coeff, title):
    """    
    Args:
        trajectory_df: DataFrame with trajectory data.
        cauchy_coeff: Cauchy coefficient for the source distribution.
        title: Title for the plot.
    """
    # Calculate turning angles
    t_angle = turning_angles(trajectory_df)
    
    # Generate source wrapped Cauchy distribution
    angles = np.linspace(-np.pi, np.pi, 1000)
    source_dist = wrapcauchy.pdf(angles, c=cauchy_coeff)
    
    # Create histogram of observed turning angles
    hist, bin_edges = np.histogram(t_angle, bins=50, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    fig = go.Figure()
    
    # Observed distribution (histogram)
    fig.add_trace(go.Bar(
        x=bin_centers,
        y=hist,
        name='Observed Distribution',
        opacity=0.6
    ))
    
    # Source distribution (curve)
    fig.add_trace(go.Scatter(
        x=angles,
        y=source_dist,
        name='Source Distribution',
        mode='lines',
        line=dict(color='red', width=2)
    ))
    
    fig.update_layout(
        title=title,
        xaxis_title='Turning Angle (radians)',
        yaxis_title='Density',
        template='plotly_dark',
        barmode='overlay'
    )
    
    fig.show()